In [1]:
import requests
import pandas as pd
import numpy as np
from multiprocessing.dummy import Pool as ThreadPool

key = "fcabada09a68ee4989c8773d205528092c38287c8f21707325e92d62596b0544"

In [2]:
POOL_SIZE

NameError: name 'POOL_SIZE' is not defined

In [2]:
url = "https://min-api.cryptocompare.com/data/top/totalvolfull?limit=100&tsym=USD"
page = requests.get(url)
good_names = np.array([i['CoinInfo']['Name'] for i in page.json()['Data']])
good_names.shape

(100,)

In [3]:
def get_json_min(title):
    url = "https://min-api.cryptocompare.com/data/v2/histominute?fsym={}&tsym=USD&limit=1440&api_key={}".format(title, key)
    page = requests.get(url)
    if "Data" in page.json()["Data"]:
        return page.json()["Data"]["Data"]
    
def get_json_hour(title):
    url = "https://min-api.cryptocompare.com/data/v2/histohour?fsym={}&tsym=USD&limit=168&api_key={}".format(title, key)
    page = requests.get(url)
    if "Data" in page.json()["Data"]:
        return page.json()["Data"]["Data"]
    
def get_json_day(title):
    url = "https://min-api.cryptocompare.com/data/v2/histoday?fsym={}&tsym=USD&limit=60&api_key={}".format(title, key)
    page = requests.get(url)
    if "Data" in page.json()["Data"]:
        return page.json()["Data"]["Data"] 

def make_clean_df(content, title):
    df = pd.DataFrame(content).rename(columns={"close": title})
    columns_drop = df.columns[[1, 2, 3, 4, 5, 7, 8]]
    df_clean = df.drop(columns_drop, axis=1)
    df_clean.set_index("time", inplace=True)
    
    return df_clean

In [4]:
def make_data(func):
    with ThreadPool(15) as pool:
        contents = pool.map(func, good_names)

    train_df = pd.DataFrame()

    for i, title in enumerate(good_names):
        if contents[i] != None:
            tmp = make_clean_df(contents[i], title)
            train_df = pd.concat([train_df, tmp], axis=1)
    return train_df

In [6]:
df_min = make_data(get_json_min)
print(df_min.shape)
df_hour = make_data(get_json_hour)
print(df_hour.shape)
df_day = make_data(get_json_day)
print(df_day.shape)

(1441, 99)
(169, 99)
(61, 99)


In [11]:
df_min.to_csv("min.csv")
df_hour.to_csv("hour.csv")
df_day.to_csv("day.csv")